In [ ]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import layers

#tf.config.run_functions_eagerly(True)
class CustomCRF(tf.keras.layers.Layer):
    def __init__(self, num_tags, seq_len, **kwargs):
        super(CustomCRF, self).__init__(**kwargs)
        self.num_tags = num_tags
        self.seq_len = seq_len

    def build(self, input_shape):
        xavier_initializer = tf.keras.initializers.GlorotUniform()
        self.start_transitions = self.add_weight(
            shape=(self.num_tags,), initializer=xavier_initializer, trainable=True
        )
        self.transition_matrix = self.add_weight(
            shape=(self.num_tags, self.num_tags), initializer=xavier_initializer, trainable=True
        )
        self.end_transitions = self.add_weight(
            shape=(self.num_tags,), initializer=xavier_initializer, trainable=True
        )

    @tf.function
    def call(self, inputs, labels=None, training=None):
        emissions, attention_mask = inputs
        seq_len = tf.shape(emissions)[1]

        if training:
            return self._crf_loss(emissions, labels, attention_mask)
        else:
            return self.viterbi(emissions, attention_mask)

    @tf.function
    def _crf_loss(self, emissions, labels, attention_mask, seq_len):
        log_likelihood = self._forward_algorithm(emissions, seq_len)
        gold_score = self._score_sequence(emissions, labels, seq_len)
        return tf.reduce_mean(log_likelihood - gold_score)

    @tf.function
    def _forward_algorithm(self, emissions, seq_len):
        batch_size, num_tags = tf.shape(emissions)[0], self.num_tags

        log_alpha = self.start_transitions + emissions[:, 0, :]

        def step_fn(t, log_alpha):
            log_alpha_expanded = tf.expand_dims(log_alpha, axis=2)
            transition_scores = tf.expand_dims(self.transition_matrix, axis=0)
            log_alpha_t = tf.reduce_logsumexp(log_alpha_expanded + transition_scores, axis=1)
            new_log_alpha = log_alpha_t + emissions[:, t, :]
            return t + 1, new_log_alpha

        t = tf.constant(1)
        _, log_alpha = tf.while_loop(
            cond=lambda t, *_: t < seq_len,
            body=step_fn,
            loop_vars=[t, log_alpha]
        )

        log_alpha += self.end_transitions
        return tf.reduce_logsumexp(log_alpha, axis=1)

    @tf.function
    def _score_sequence(self, emissions, labels, seq_len):
        batch_size = tf.shape(labels)[0]

        start_label_indices = labels[:, 0]
        start_transition_scores = tf.gather(self.start_transitions, start_label_indices)

        first_emission_scores = tf.gather_nd(
            emissions,
            indices=tf.stack(
                [tf.range(batch_size, dtype=tf.int64), tf.zeros(batch_size, dtype=tf.int64), start_label_indices],
                axis=1
            )
        )

        score = start_transition_scores + first_emission_scores

        for t in range(1, seq_len):
            prev_labels = labels[:, t - 1]
            curr_labels = labels[:, t]

            indices = tf.stack([prev_labels, curr_labels], axis=1)
            transition_scores = tf.gather_nd(self.transition_matrix, indices)
            curr_emission_scores = tf.gather_nd(
                emissions,
                indices=tf.stack(
                    [tf.range(batch_size, dtype=tf.int64), tf.cast(tf.fill([batch_size], t), tf.int64), curr_labels],
                    axis=1
                )
            )

            score += transition_scores + curr_emission_scores

        end_label_indices = labels[:, seq_len - 1]
        end_transition_scores = tf.gather(self.end_transitions, end_label_indices)

        score += end_transition_scores

        return score

    @tf.function
    def viterbi(self, emissions, attention_mask):
        batch_size = tf.shape(emissions)[0]
        seq_len = tf.shape(emissions)[1]
        num_tags = tf.shape(emissions)[2]

        dp = tf.TensorArray(dtype=tf.float32, size=seq_len, clear_after_read=False)
        backpointer = tf.TensorArray(dtype=tf.int32, size=seq_len, clear_after_read=False)

        first_step = self.start_transitions + emissions[:, 0, :]
        dp = dp.write(0, first_step)

        t = tf.constant(1)

        def loop_body(t, dp, backpointer):
            prev_scores = dp.read(t - 1)
            scores = tf.expand_dims(prev_scores, axis=2) + self.transition_matrix
            best_scores = tf.reduce_max(scores, axis=1)
            best_paths = tf.argmax(scores, axis=1, output_type=tf.int32)
            current_scores = emissions[:, t, :] + best_scores
            dp = dp.write(t, current_scores)
            backpointer = backpointer.write(t, best_paths)
            return t + 1, dp, backpointer

        _, dp, backpointer = tf.while_loop(
            cond=lambda t, *_: t < seq_len,
            body=loop_body,
            loop_vars=(t, dp, backpointer)
        )

        last_step_scores = dp.read(seq_len - 1) + self.end_transitions
        last_tag = tf.argmax(last_step_scores, axis=1, output_type=tf.int32)

        def backtrack_fn(i):
            best_path = tf.TensorArray(dtype=tf.int32, size=seq_len, clear_after_read=False)
            best_path = best_path.write(seq_len - 1, last_tag[i])
            t = seq_len - 2

            def backtrack_body(t, best_path):
                next_tag = best_path.read(t + 1)
                best_tag = backpointer.read(t + 1)[i, next_tag]
                best_path = best_path.write(t, best_tag)
                return t - 1, best_path

            _, best_path = tf.while_loop(
                cond=lambda t, *_: t >= 0,
                body=backtrack_body,
                loop_vars=(t, best_path)
            )

            return best_path.stack()

        best_paths = tf.map_fn(backtrack_fn, tf.range(batch_size), fn_output_signature=tf.int32)
        return best_paths


class NERREModel(tf.keras.Model):
    def __init__(self, num_tags, hidden_dim, seq_len, num_relations, ner_lr=1e-3, re_lr=1e-4, **kwargs):
        super(NERREModel, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained('bert-base-uncased')
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim, return_sequences=True))
        self.lstm.build((None, None, self.bert.config.hidden_size))

        self.dense_ner = tf.keras.layers.Dense(num_tags)
        self.dense_ner.build((None, 2*hidden_dim))

        self.crf = CustomCRF(num_tags, seq_len)

        self.dense1_re = tf.keras.layers.Dense(384, activation='relu')
        self.dense1_re.build((None, 4*hidden_dim))
        self.dense2_re = tf.keras.layers.Dense(128, activation='relu')
        self.dense2_re.build((None, 384))
        self.dense3_re = tf.keras.layers.Dense(num_relations, activation='softmax')
        self.dense3_re.build((None, 128))

        self.ner_optimizer = tf.keras.optimizers.Adam(learning_rate=ner_lr)
        self.re_optimizer = tf.keras.optimizers.Adam(learning_rate=re_lr)
        self.num_tags = num_tags
        self.num_relations = num_relations
        self.ner_metrics_dict = {i: tf.Variable([0, 0, 0, 0], dtype=tf.int32) for i in range(num_tags)}
        self.re_metrics_dict = {i: tf.Variable([0, 0, 0, 0], dtype=tf.int32) for i in range(num_relations)}

    def build(self, input_shape):
        self.crf.build(input_shape)

    def reset_metrics(self):
        for key in self.re_metrics_dict:
            self.re_metrics_dict[key].assign([0, 0, 0, 0])
        for key in self.ner_metrics_dict:
            self.ner_metrics_dict[key].assign([0, 0, 0, 0])

    @tf.function
    def extract_entity_pairs(self, lstm_output, ner_tags, re_mask):
        combined_embeddings = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        batch_size = tf.shape(ner_tags)[0]

        def process_batch(i, embeddings_list):
            re_pairs = re_mask[i]
            valid_pairs = tf.boolean_mask(re_pairs, tf.not_equal(re_pairs[:, 0], -1))

            def process_pair(j, embeddings_list):
                e1_sidx, e1_eidx, relation_type, e2_sidx, e2_eidx = tf.unstack(valid_pairs[j])
                e1_emb = self.pool_entity(lstm_output[i], ner_tags[i], e1_sidx, e1_eidx)
                e2_emb = self.pool_entity(lstm_output[i], ner_tags[i], e2_sidx, e2_eidx)
                combined = tf.concat([e1_emb, e2_emb], axis=-1)
                return j + 1, embeddings_list.write(embeddings_list.size(), combined)

            _, embeddings_list = tf.while_loop(
                lambda j, _: j < tf.shape(valid_pairs)[0],
                process_pair,
                loop_vars=[0, embeddings_list]
            )
            return i + 1, embeddings_list

        _, final_embeddings = tf.while_loop(
            lambda i, _: i < batch_size,
            process_batch,
            loop_vars=[0, combined_embeddings]
        )

        return final_embeddings.stack()
    @tf.function
    def pool_entity(self, lstm_output, ner_tags, start_idx, end_idx):
        entity_span = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)

        def condition(idx, _):
            return idx <= end_idx

        def body(idx, entity_span):
            entity_span = entity_span.write(entity_span.size(), lstm_output[idx])
            return idx + 1, entity_span

        _, collected_span = tf.while_loop(
            condition, body, loop_vars=[start_idx, entity_span]
        )

        return tf.reduce_mean(collected_span.stack(), axis=0)

    @tf.function
    def extract_relation_labels(self, re_mask):
        reduced_re_mask = re_mask[:, :, 2]
        valid_mask = tf.not_equal(reduced_re_mask, -1)
        valid_relation_labels = tf.boolean_mask(reduced_re_mask, valid_mask)
        return valid_relation_labels
    @tf.function
    def call(self, inputs, labels=None, training=None):
        bert_output = self.bert(inputs['input_ids'], attention_mask=inputs['attention_mask'])['last_hidden_state']
        lstm_output = self.lstm(bert_output)
        ner_logits = self.dense_ner(lstm_output)
        re_embeddings = self.extract_entity_pairs(lstm_output, inputs['ner_tags'], inputs['re_mask'])
        combined_embeddings_stacked = tf.stack(re_embeddings)

        re_logits = self.dense3_re(self.dense2_re(self.dense1_re(combined_embeddings_stacked)))

        if training:
            return ner_logits, re_logits
        else:
            return self.crf((ner_logits, inputs['attention_mask']), training=False), re_logits
    @tf.function
    def train_step(self, data):
        inputs, labels = data

        with tf.GradientTape(persistent=True) as tape:
            ner_logits, re_logits = self(inputs, training=True)
            ner_loss = self.crf._crf_loss(ner_logits, labels, inputs['attention_mask'], tf.shape(ner_logits)[1])
            filtered_relation_labels = self.extract_relation_labels(inputs['re_mask'])
            re_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
                filtered_relation_labels, re_logits, from_logits=False
            ))

        ner_trainable_vars = [self.lstm.trainable_variables, self.dense_ner.trainable_variables, self.crf.trainable_variables]
        ner_trainable_vars = [var for sublist in ner_trainable_vars for var in sublist]
        re_trainable_vars = [self.lstm.trainable_variables, self.dense1_re.trainable_variables, self.dense2_re.trainable_variables, self.dense3_re.trainable_variables]
        re_trainable_vars = [var for sublist in re_trainable_vars for var in sublist]

        ner_gradients = tape.gradient(ner_loss, ner_trainable_vars)
        self.ner_optimizer.apply_gradients(zip(ner_gradients, ner_trainable_vars))
        re_gradients = tape.gradient(re_loss, re_trainable_vars)
        self.re_optimizer.apply_gradients(zip(re_gradients, re_trainable_vars))

        del tape

        return {"ner_loss": ner_loss, "re_loss": re_loss}
    def test_step(self, data, validation = False):
        inputs, labels = data

        if validation:
            ner_logits, re_logits = self(inputs, training=True)
            ner_loss = self.crf._crf_loss(ner_logits, labels, inputs['attention_mask'], tf.shape(ner_logits)[1])
            filtered_relation_labels = self.extract_relation_labels(inputs['re_mask'])
            re_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(
                filtered_relation_labels, re_logits, from_logits=False
            ))
            return {"ner_loss": ner_loss, "re_loss": re_loss}

        emissions, re_logits = self(inputs, training=False)
        predictions = tf.reshape(emissions, [-1])
        true_labels = tf.reshape(labels, [-1])

        for i in range(self.num_tags):
            true_positives = tf.reduce_sum(tf.cast((predictions == i) & (true_labels == i), tf.int32))
            false_positives = tf.reduce_sum(tf.cast((predictions == i) & (true_labels != i), tf.int32))
            false_negatives = tf.reduce_sum(tf.cast((predictions != i) & (true_labels == i), tf.int32))
            true_negatives = tf.reduce_sum(tf.cast((predictions != i) & (true_labels != i), tf.int32))

            self.ner_metrics_dict[i].assign_add([true_positives, true_negatives, false_positives, false_negatives])

        predictions = tf.argmax(re_logits, axis=-1)
        true_labels = self.extract_relation_labels(inputs['re_mask'])

        for i in range(self.num_relations):
            tp = tf.reduce_sum(tf.cast((predictions == i) & (true_labels == i), tf.int32))
            fp = tf.reduce_sum(tf.cast((predictions == i) & (true_labels != i), tf.int32))
            fn = tf.reduce_sum(tf.cast((predictions != i) & (true_labels == i), tf.int32))
            tn = tf.reduce_sum(tf.cast((predictions != i) & (true_labels != i), tf.int32))

            self.re_metrics_dict[i].assign_add([tp, tn, fp, fn])


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import random
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
ds = load_dataset("bigbio/chemprot", "chemprot_full_source")

def gen(i, j, ner_tags_segment, next_tags):
    l, m = i + 1, j + 1
    tem = []
    while ner_tags_segment[l] in next_tags[ner_tags_segment[i]][0]:
        l += 1
    while ner_tags_segment[m] in next_tags[ner_tags_segment[j]][0]:
        m += 1
    if m - 1 != j and l - 1 != i:
        tem.append((i, l - 1, j, m - 1))
    l, m = i + 1, j + 1
    while ner_tags_segment[l] in next_tags[ner_tags_segment[i]][0]:
        l += 1
    while ner_tags_segment[m] in next_tags[ner_tags_segment[j]][1]:
        m += 1
    if m - 1 != j and l - 1 != i:
        tem.append((i, l - 1, j, m - 1))
    l, m = i + 1, j + 1
    while ner_tags_segment[l] in next_tags[ner_tags_segment[i]][1]:
        l += 1
    while ner_tags_segment[m] in next_tags[ner_tags_segment[j]][0]:
        m += 1
    if m - 1 != j and l - 1 != i:
        tem.append((i, l - 1, j, m - 1))
    l, m = i + 1, j + 1
    while ner_tags_segment[l] in next_tags[ner_tags_segment[i]][1]:
        l += 1
    while ner_tags_segment[m] in next_tags[ner_tags_segment[j]][1]:
        m += 1
    if m - 1 != j and l - 1 != i:
        tem.append((i, l - 1, j, m - 1))
    return tem

def prepare_data(row, tag_dict, r_dict, tag_dict_map, next_tags, max_length=128, stride=10, padding=True):
    text = row['text']
    entities = row['entities']
    relations = row['relations']

    tokenized_inputs = tokenizer(
        text,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        max_length=max_length,
        truncation=True,
        stride=stride,
        padding='max_length' if padding else False
    )

    input_ids = []
    attention_mask = []
    ner_tags = []
    relation_mask = []

    for i, offset_mapping in enumerate(tokenized_inputs['offset_mapping']):
        input_ids_segment = tokenized_inputs['input_ids'][i]
        attention_mask_segment = tokenized_inputs['attention_mask'][i]

        ner_tags_segment = [(0,) for i in range(max_length)]
        rel_mask_segment = []
        entity_map = {}
        b_tag_indices = []

        for entity_id, entity_type, (start_char, end_char) in zip(entities['id'], entities['type'], entities['offsets']):
            entity_start_found = False
            for token_idx, (token_start, token_end) in enumerate(offset_mapping):
                if token_idx == 0 or token_idx == max_length - 1:
                    continue
                if token_start == start_char and offset_mapping[-2][1] >= end_char:
                    ner_tags_segment[token_idx] += (tag_dict[f'B-{entity_type}'],)
                    entity_map[entity_id] = [token_idx, token_idx]
                    entity_start_found = True
                elif entity_start_found and token_start < end_char:
                    ner_tags_segment[token_idx] += (tag_dict[f'I-{entity_type}'],)
                    entity_map[entity_id][1] = token_idx

        rel_pairs = set()
        for rel_type, arg1_id, arg2_id in zip(relations['type'], relations['arg1'], relations['arg2']):
            if arg1_id in entity_map and arg2_id in entity_map:
                rel_mask_segment.append([entity_map[arg1_id][0], entity_map[arg1_id][1], r_dict[rel_type], entity_map[arg2_id][0], entity_map[arg2_id][1]])
                rel_pairs.add((entity_map[arg1_id][0], entity_map[arg1_id][1], entity_map[arg2_id][0], entity_map[arg2_id][1]))

        for j in range(max_length):
            ner_tags_segment[j] = tag_dict_map[tuple(sorted(list(set(ner_tags_segment[j]))))]
            if ner_tags_segment[j] in {1, 2, 4, 7, 8, 12, 13, 15, 16, 17, 18}:
                b_tag_indices.append(j)

        relpairs = set(rel_pairs)
        neg_samples = []
        for i in rel_pairs:
            for j in b_tag_indices:
                tem = gen(i[0], j, ner_tags_segment, next_tags)
                for ke in tem:
                    if ke not in relpairs:
                        relpairs.add(ke)
                        neg_samples.append([ke[0], ke[1], 11, ke[2], ke[3]])
                tem = gen(j, i[2], ner_tags_segment, next_tags)
                for ke in tem:
                    if ke not in relpairs:
                        relpairs.add(ke)
                        neg_samples.append([ke[0], ke[1], 11, ke[2], ke[3]])

        neg_samples = random.sample(neg_samples, len(rel_pairs)) if len(rel_pairs) != 0 and len(rel_pairs) <= len(neg_samples) else random.sample(neg_samples, 15) if len(neg_samples) >= 15 else neg_samples

        neg_samples_r = []
        for i in b_tag_indices:
            for j in b_tag_indices:
                tem = gen(i, j, ner_tags_segment, next_tags)
                for ke in tem:
                    if ke not in relpairs:
                        relpairs.add(ke)
                        neg_samples_r.append([ke[0], ke[1], 11, ke[2], ke[3]])

        neg_samples_r = random.sample(neg_samples_r, len(rel_pairs)) if len(rel_pairs) != 0 and len(rel_pairs) <= len(neg_samples_r) else random.sample(neg_samples_r, 15) if len(neg_samples_r) >= 15 else neg_samples_r
        input_ids.append(input_ids_segment)
        attention_mask.append(attention_mask_segment)
        ner_tags.append(ner_tags_segment)
        relation_mask.append(rel_mask_segment + neg_samples + neg_samples_r)

    return input_ids, attention_mask, ner_tags, relation_mask



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

(…)prot_full_source/validation/0000.parquet:   0%|          | 0.00/727k [00:00<?, ?B/s]

Generating sample split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1020 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/612 [00:00<?, ? examples/s]

In [ ]:
tag_dict = {
    'B-GENE-Y': 1, 'I-GENE-Y': 2,
    'B-GENE-N': 3, 'I-GENE-N': 4,
    'B-CHEMICAL': 5, 'I-CHEMICAL': 6
}

tag_dict_map = {
    (0,): 0, (0, 1): 1, (0, 2, 3, 5): 2,
    (0, 4): 3, (0, 1, 6): 4, (0, 2, 6): 5, (0, 2): 6, (0, 3, 6): 7,
    (0, 5): 8, (0, 4, 6): 9, (0, 2, 4, 6): 10, (0, 2, 4): 11,
    (0, 5, 6): 12, (0, 3): 13, (0, 6): 14, (0, 1, 5): 15, (0, 2, 5): 16, (0, 3, 5): 17, (0, 4, 5): 18
}

con = {2:[2,5,6,10,11,16], 4:[3,9,10,11,18], 6:[4,5,7,9,10,12,14]}

next_entity = {0:[[0],[]], 1:[con[2],[]], 2:[con[4],con[6]], 3:[[3],[]], 4:[con[2],[]], 5:[[5],[]], 6:[[6],[]], 7:[con[4],[]], 8:[con[6],[]], 9:[[9],[]] ,10:[[10],[]], 11:[[11],[]], 12:[con[6],[]], 13:[con[4],[]], 14:[[14],[]], 15:[con[2],con[6]], 16:[con[6],[]], 17:[con[4],con[6]], 18:[con[6],[]]}

r_dict = {
    'CPR:0': 0, 'CPR:1': 1, 'CPR:2': 2, 'CPR:3': 3,
    'CPR:4': 4, 'CPR:5': 5, 'CPR:6': 6, 'CPR:7': 7,
    'CPR:8': 8, 'CPR:9': 9, 'CPR:10': 10
}

all_input_ids, all_attention_masks, all_ner_tags, all_relation_masks = [], [], [], []
for row in ds['train']:
    input_ids, attention_mask, ner_tags, relation_mask = prepare_data(row, tag_dict, r_dict, tag_dict_map, next_entity)
    all_input_ids.extend(input_ids)
    all_attention_masks.extend(attention_mask)
    all_ner_tags.extend(ner_tags)
    all_relation_masks.extend(relation_mask)

prepared_data = {
    'input_ids': np.array(all_input_ids),
    'attention_mask': np.array(all_attention_masks),
    'ner_tags': np.array(all_ner_tags),
    're_mask': all_relation_masks
}

hidden_dim = 128
num_tags = 19
seq_len = 128
num_relations = 12

padded_relation_mask = pad_sequences(
    [np.array(item, dtype=int) for item in prepared_data['re_mask']],
    padding='post',
    value=-1,
    dtype='object'
)

padded_relation_mask = np.array(padded_relation_mask, dtype=int)

inputs = {
    'input_ids': prepared_data['input_ids'],
    'attention_mask': prepared_data['attention_mask'],
    're_mask': padded_relation_mask,
    'ner_tags': prepared_data['ner_tags']
}

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        're_mask': padded_relation_mask,
        'ner_tags': inputs['ner_tags']
    },
    inputs['ner_tags']
))


vall_input_ids, vall_attention_masks, vall_ner_tags, vall_relation_masks = [], [], [], []
for row in ds['validation']:
    input_ids, attention_mask, ner_tags, relation_mask = prepare_data(row, tag_dict, r_dict, tag_dict_map, next_entity)
    vall_input_ids.extend(input_ids)
    vall_attention_masks.extend(attention_mask)
    vall_ner_tags.extend(ner_tags)
    vall_relation_masks.extend(relation_mask)

vprepared_data = {
    'input_ids': np.array(vall_input_ids),
    'attention_mask': np.array(vall_attention_masks),
    'ner_tags': np.array(vall_ner_tags),
    're_mask': vall_relation_masks
}


vpadded_relation_mask = pad_sequences(
    [np.array(item, dtype=int) for item in vprepared_data['re_mask']],
    padding='post',
    value=-1,
    dtype='object'
)

vpadded_relation_mask = np.array(vpadded_relation_mask, dtype=int)

vinputs = {
    'input_ids': vprepared_data['input_ids'],
    'attention_mask': vprepared_data['attention_mask'],
    're_mask': vpadded_relation_mask,
    'ner_tags': vprepared_data['ner_tags']
}

vdataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': vinputs['input_ids'],
        'attention_mask': vinputs['attention_mask'],
        're_mask': vpadded_relation_mask,
        'ner_tags': vinputs['ner_tags']
    },
    vinputs['ner_tags']
))


In [ ]:

nerre_model = NERREModel(num_tags, hidden_dim, seq_len, num_relations)
nerre_model.compile()
batch_size = 30
dataset = dataset.batch(batch_size)
vdataset = vdataset.batch(batch_size)
num_epochs = 20
num_batches = 131

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_ner_loss = 0
    val_ner_loss = 0
    train_re_loss = 0
    val_re_loss = 0

    t_b, v_b = 0, 0
    for batch_num, batch_data in enumerate(dataset.take(num_batches)):
        metrics = nerre_model.train_step(batch_data)
        train_ner_loss += metrics['ner_loss']
        train_re_loss += metrics['re_loss']
        t_b += 1
    for batch_num, batch_data in enumerate(vdataset.take(num_batches)):
        metrics = nerre_model.test_step(batch_data, validation = True)
        val_ner_loss += metrics['ner_loss']
        val_re_loss += metrics['re_loss']
        v_b += 1

    print(f"End of Epoch {epoch + 1}, train NER loss: {train_ner_loss / t_b} train RE loss: {train_re_loss / t_b} validation NER loss: {val_ner_loss / v_b} validation RE loss: {val_re_loss / v_b}")

In [ ]:

all_input_ids, all_attention_masks, all_ner_tags, all_relation_masks = [], [], [], []
for row in ds['test']:
    input_ids, attention_mask, ner_tags, relation_mask = prepare_data(row, tag_dict, r_dict, tag_dict_map, next_entity)
    all_input_ids.extend(input_ids)
    all_attention_masks.extend(attention_mask)
    all_ner_tags.extend(ner_tags)
    all_relation_masks.extend(relation_mask)

prepared_data = {
    'input_ids': np.array(all_input_ids),
    'attention_mask': np.array(all_attention_masks),
    'ner_tags': np.array(all_ner_tags),
    're_mask': all_relation_masks
}

padded_relation_mask = pad_sequences(
    [np.array(item, dtype=int) for item in prepared_data['re_mask']],
    padding='post',
    value=-1,
    dtype='object'
)

padded_relation_mask = np.array(padded_relation_mask, dtype=int)

inputs = {
    'input_ids': prepared_data['input_ids'],
    'attention_mask': prepared_data['attention_mask'],
    're_mask': padded_relation_mask,
    'ner_tags': prepared_data['ner_tags']
}

In [ ]:
nerre_model.reset_metrics()
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        're_mask': padded_relation_mask,
        'ner_tags': inputs['ner_tags']
    },
    inputs['ner_tags']
))
dataset = dataset.batch(batch_size)

for batch_num, batch_data in enumerate(dataset.take(num_batches)):
    nerre_model.test_step(batch_data)





In [ ]:
model_save_path = '/content/nerre_model.keras'
nerre_model.save(model_save_path)

from google.colab import files
files.download(model_save_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

def calculate_final_metrics(num_classes, threshold, metrics_dict):
    avg_precision, avg_recall, avg_f1_score = [], [], []
    total_tp, total_fp, total_fn, total_tn = 0, 0, 0, 0
    tpa,fpa,tna,fna,k=[],[],[],[],0
    for i in range(num_classes):
        tp, tn, fp, fn = metrics_dict[i].numpy()
        if tp+fn < threshold:
            continue

        k += 1
        tpa.append(tp)
        fpa.append(fp)
        tna.append(tn)
        fna.append(fn)

        total_tp += tp
        total_fp += fp
        total_fn += fn
        total_tn += tn

        precision = tp / (tp + fp + tf.keras.backend.epsilon())
        recall = tp / (tp + fn + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

        avg_precision.append(precision)
        avg_recall.append(recall)
        avg_f1_score.append(f1)

    accuracy = (total_tp ) / (total_tp + total_fp + tf.keras.backend.epsilon())
    return {
        "average_precision": tf.reduce_mean(avg_precision),
        "average_recall": tf.reduce_mean(avg_recall),
        "average_f1_score": tf.reduce_mean(avg_f1_score),
        "accuracy": accuracy
    }


In [ ]:
print('NER task metrics')
ner_metrics = calculate_final_metrics(19, 10, nerre_model.ner_metrics_dict)
for key in ner_metrics.keys():
    print(f"{key}: {ner_metrics[key]}")

In [ ]:
print('RE task metrics')
re_metrics = calculate_final_metrics(12, 30, nerre_model.re_metrics_dict)
for key in re_metrics.keys():
    print(f"{key}: {re_metrics[key]}")
